Importacion de librerias y Configuracion

In [38]:
import polars as pl
from polars import Config
Config.set_fmt_str_lengths(100)

polars.config.Config

BASE TICKETS

In [9]:
dfTicketsHistorico=pl.read_csv(
    source="Tickets\Tickets Historico.txt",
    separator=";",
    columns=["Numero Ticket","Ubicacion","Service Desk","Estado","Fecha Creacion","Fecha Termino","Fecha Cierre"],
    try_parse_dates=True, #Intenta convertir las fechas a yyyy-MM-dd
    ignore_errors=True
).rename({"Numero Ticket": "TicketID"})

In [39]:
dfTicketsActual=pl.read_csv(
    source="Tickets\Tickets Historico.txt",
    separator=";",
    columns=["Numero Ticket","Ubicacion","Service Desk","Estado","Fecha Creacion","Fecha Termino","Fecha Cierre"],
    try_parse_dates=True, #Intenta convertir las fechas a yyyy-MM-dd
    ignore_errors=True
).rename({"Numero Ticket": "TicketID"})

In [40]:
dfTicketsActual=dfTicketsActual.filter(
    pl.col("TicketID").str.starts_with("WO")
)

In [41]:
dfTicket=pl.concat([dfTicketsHistorico,dfTicketsActual],how="vertical")


In [44]:
dfTicket = dfTicket.sort(
    by=["TicketID","Fecha Creacion"],
    descending=[True,True]
).unique(
    subset="TicketID",
    keep="last",
    maintain_order=True
)

In [48]:
dfTicket = dfTicket.with_columns(
    pl.col("Ubicacion")
    .str.split_exact(" - ",1)
    .struct.rename_fields(["Agencia","AgenciaID"]) 
).unnest("Ubicacion")

In [49]:
dfTicket = dfTicket.cast({"AgenciaID": pl.Int64})

In [50]:
dfTicket=dfTicket.with_columns(
    pl.when(pl.col("Fecha Termino").is_null())
    .then(pl.col("Fecha Cierre"))
    .otherwise(pl.col("Fecha Termino"))
    .alias("Fecha Real Fin")
)

In [51]:
dfTicket=dfTicket.with_columns(
    (pl.col("Fecha Real Fin") - pl.col("Fecha Creacion"))
    .dt.total_days()
    .alias("Dias Cierre")
)

In [61]:
dfTicket = dfTicket.with_columns(    
    pl.when(pl.col("Dias Cierre").is_null()).then(None)
    .when(pl.col("Dias Cierre") <= 3).then(pl.lit("0 a 3 dias"))
    .when(pl.col("Dias Cierre") <= 7).then(pl.lit("4 a 7 dias"))
    .when(pl.col("Dias Cierre") <= 15).then(pl.lit("8 a 15 dias"))    
    .otherwise(pl.lit("+15 dias"))
    .alias("Grupo Dias")
)

Resultado esperado

In [67]:
dfTicket.select(
    pl.col("TicketID"),
    pl.col("Service Desk"),
    pl.col("Estado"),
    pl.col("Fecha Creacion"),
    pl.col("Fecha Real Fin"),
    pl.col("Agencia"),
    pl.col("AgenciaID"),
    pl.col("Dias Cierre"),
    pl.col("Grupo Dias")
).head(10)

TicketID,Service Desk,Estado,Fecha Creacion,Fecha Real Fin,Agencia,AgenciaID,Dias Cierre,Grupo Dias
str,str,str,date,date,str,i64,i64,str
"""WO0000004849732""","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-06,"""EL PINO""",191087,6,"""4 a 7 dias"""
"""WO0000004849726""","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-02,"""LUREN""",380004,2,"""0 a 3 dias"""
"""WO0000004849721""","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-02,"""LUREN""",380004,2,"""0 a 3 dias"""
"""WO0000004849718""","""Zona Norte""","""Rechazado""",2022-12-31,null,"""AREQUIPA""",215000,null,null
"""WO0000004849382""","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-04,"""LOS DAMASCOS""",193038,4,"""4 a 7 dias"""
"""WO0000004849379""","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-25,"""NAYLAMP""",191143,25,"""+15 dias"""
"""WO0000004849355""","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-03,"""VITARTE""",191043,3,"""0 a 3 dias"""
"""WO0000004849330""","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-14,"""IMPERIAL""",255001,14,"""8 a 15 dias"""
"""WO0000004849307""","""Zona Centro""","""Rechazado""",2022-12-31,null,"""TUSILAGOS""",191108,null,null


BASE ATENCIONES

In [72]:
#Leer los archivos que estan dentro de una carpeta
from pathlib import Path
rutaCarpeta = Path("Atenciones/")
archivosExcel=list(rutaCarpeta.glob("Atenciones*xlsx"))
archivosExcel

[WindowsPath('Atenciones/Atenciones Centro.xlsx'),
 WindowsPath('Atenciones/Atenciones Norte.xlsx'),
 WindowsPath('Atenciones/Atenciones Sur.xlsx')]

In [79]:
dfAtenciones = pl.DataFrame()

for rutaarchivo in archivosExcel:
    archivo= str(rutaarchivo).split("\\")[1]
    aux_DF = pl.read_excel(
        source="Atenciones/"+archivo,
        sheet_name="Hoja1",
        engine="xlsx2csv",
        read_csv_options={
            "columns":["Numero Ticket","Tipo de Ticket","Proveedor","Costo Atencion"],
            "infer_schema_length":0
            }
    ).rename({"Numero Ticket": "TicketID"})
    aux_DF = aux_DF.with_columns(pl.lit(archivo).alias("Nombre Archivo"))
    dfAtenciones = pl.concat([dfAtenciones,aux_DF],how="vertical")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3908\2290268327.py:5: DeprecationWarning: `the argument read_csv_options` for `read_excel` is deprecated. It has been renamed to `read_options`.
  aux_DF = pl.read_excel(


In [85]:
dfAtenciones = dfAtenciones.select(
    pl.col("TicketID"),
    pl.col("Tipo de Ticket"),
    pl.col("Proveedor"),
    pl.col("Costo Atencion").str.strip_chars()
    .str.to_uppercase()
    .str.replace_many(["COSTO CERO","SIN COSTO"],"0")
)

In [93]:
def texto_a_decimal(valor):
    try:
        return round(float(valor),2)
    except:
        return None

In [95]:
dfAtenciones = dfAtenciones.select(
    pl.col("TicketID"),
    pl.col("Tipo de Ticket"),
    pl.col("Proveedor"),
    pl.col("Costo Atencion").map_elements(texto_a_decimal)
)

Resultado Esperado

In [99]:
dfAtenciones.head(10)

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,f64
"""WO0000005584689""","""Variable""","""MR EXPRESS""",null
"""WO0000004544943""","""Rechazado""",null,100.0
"""WO0000004545246""","""Rechazado""",null,100.0
"""WO0000004559125""","""Rechazado""",null,100.0
"""WO0000004707477""","""Rechazado""",null,100.0
"""WO0000004759976""","""Rechazado""",null,100.0
"""WO0000004795876""","""Rechazado""",null,100.0
"""WO0000004825300""","""Rechazado""",null,100.0
"""WO0000004853680""","""Flat""","""ACCENTURE""",100.0


COMBINAR Y EXPORTAR

In [105]:
dfTicketAtenciones = dfTicket.join(
    dfAtenciones,
    on = "TicketID",
    how= "inner"
).select(
    "TicketID",
    "AgenciaID",
    "Agencia",
    "Service Desk",
    "Estado",
    "Fecha Creacion",
    "Fecha Real Fin",
    "Grupo Dias",
    "Tipo de Ticket",
    "Costo Atencion"
).rename({"Fecha Real Fin":"Fecha Cierre","Tipo de Ticket":"Tipo Ticket","Costo Atencion":"Costo"})

In [107]:
#Exportando a un excel
dfTicketAtenciones.write_excel(
    workbook="Consolidado.xlsx",
    worksheet="Atenciones",
    table_style="Table Style Light 2",
    dtype_formats={pl.Date: "dd/mm/yyyy"},
    float_precision=2,
    autofit=True
)